In [ ]:
import tensorflow

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:28<00:00, 42.5MB/s]
100% 1.06G/1.06G [00:28<00:00, 39.7MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense , Flatten
from keras.applications.vgg16 import VGG16

In [ ]:
conv_base = VGG16(
    weights='imagenet',
    include_top=False,                   ### only including convo part of architecture excluding dense layers   (defination of transfer learning)
    input_shape=(150,150,3)
)

58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [ ]:
conv_base.trainable = False               ### freezing the convo part

In [ ]:
model.summary()                    ### non trainable parameters

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=16,
    image_size=(150,150)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=16,
    image_size=(150,150)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [ ]:
# Normalize
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
1250/1250 [==============================] - 69s 47ms/step - loss: 0.2666 - accuracy: 0.8853 - val_loss: 0.2049 - val_accuracy: 0.9072
Epoch 2/10
1250/1250 [==============================] - 68s 54ms/step - loss: 0.1893 - accuracy: 0.9188 - val_loss: 0.2079 - val_accuracy: 0.9084
Epoch 3/10
1250/1250 [==============================] - 58s 46ms/step - loss: 0.1575 - accuracy: 0.9353 - val_loss: 0.3085 - val_accuracy: 0.8838
Epoch 4/10
1250/1250 [==============================] - 58s 47ms/step - loss: 0.1296 - accuracy: 0.9456 - val_loss: 0.2260 - val_accuracy: 0.9084
Epoch 5/10
1250/1250 [==============================] - 58s 46ms/step - loss: 0.1013 - accuracy: 0.9605 - val_loss: 0.2488 - val_accuracy: 0.9028
Epoch 6/10
1250/1250 [==============================] - 57s 46ms/step - loss: 0.0745 - accuracy: 0.9704 - val_loss: 0.2524 - val_accuracy: 0.9070
Epoch 7/10
1250/1250 [==============================] - 58s 46ms/step - loss: 0.0574 - accuracy: 0.9779 - val_loss: 0.4227 -